# Chapter 5: Wrangling and Feature Engineering

Feature Engineering is that one component of the data science pipeline, that can offer a significant boost to the predictive abilities if the data is being fed to a machine learning algorithm for prediction task. It also strengthens the ability to see meaningful correlation of data points with each other. This may sound confusing for now but you'll realize how significant and challenging this step is. 

Feature Engineering is the step of manipulating existing featues or creating new ones out of the existing set of features. The raw data that we get, after clean up, has to be prepared in a form that is is ready for statistical analysis and prediction. It also enables understanding the complex relation between features. To illustrate how this creates value, suppose we have a computer-generated alpha-numeric customer ID for every customer who buys from an online store. If you were given record of every customer purchase along with that customer ID field, without any further details, this column would just be a set of strings that won't make any sense to people interpreting this data or to any algorithm no matter how powerful they are for prediction. But if you later figure out, that the first 5 digits in each customer is the Zipcode of the customer's browsing address while making a purchase, you can create a new feature of current zipcode and probably start finding patterns of how location affects purchase on the online store. 

The point of going through the example above was to tell is that some columns might contain concealed information that if revealed in form of another feature can provide more predictive capability then just a set of strings that make no sense on its own. Similarly, feature engineering also involves creating combination of features into a new one. In a social impact analysis setting, as we will see in our guided practice ahead, it will make more sense to use hit-rate (created by dividing 

### <span style="color:blue">Wrangling and Feature Engineering</span>

Emphasis on:

- How data needs to be converted to a form that prepares it for statistical analysis eg. parsing and converting reference numbers into meaningful categories.
- Similarly, converting timestamp into various temporal features like Is weekend/weekday? Month/year 
- How context specific feature formation is highly significant eg. boolean feature if stop made in intertwilight
- Categorical/String to Numeric conversion mandatory for R/Python ML functions

BPI based illustration:
- As mentioned above, conversion to boolean feature if stop made in intertwilight

**Exercise:**
- Creating intertwilight feature for BPI
- Learning how to play with date/time and conversion to various temp features
- (Optional) Use featuretools to see what combinations it creates - Careful about the automation!

Comments: Replace Veil of darkness analysis with something shown in the plots